In [5]:

def get_smallest_raster_path(benchmark_path, *candidate_paths):
    def get_raster_shape(raster_path):
        with rasterio.open(raster_path) as src:
            return src.shape

    all_paths = [benchmark_path] + list(candidate_paths)
    smallest_raster = None
    smallest_size = float('inf')

    for raster_path in all_paths:
        shape = get_raster_shape(raster_path)
        size = shape[0] * shape[1]
        if size < smallest_size:
            smallest_size = size
            smallest_raster = raster_path

    return smallest_raster

def create_shapefile_from_raster(raster_path):
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        nodata_value = src.nodata
        crs = src.crs
        
    if nodata_value is None:
        is_valid = np.ones_like(raster_data, dtype=np.uint8)
    else:
        is_valid = (raster_data != nodata_value).astype(np.uint8)

    valid_shapes = []
    for coords, value in features.shapes(is_valid, transform=transform):
        if value != 0:
            geom = shape(coords)
            valid_shapes.append({'geometry': geom, 'properties': {}})

    gdf = gpd.GeoDataFrame(valid_shapes, geometry='geometry', crs=crs)
    union_geom = unary_union(gdf.geometry)
    bounding_geom = union_geom.convex_hull
    bounding_gdf = gpd.GeoDataFrame({'geometry': [bounding_geom]}, crs=gdf.crs)

    return bounding_geom

smallest_raster_path = get_smallest_raster_path(benchmark_path, *candidate_paths)

bounding_geom=create_shapefile_from_raster(smallest_raster_path)


Bounding geometry of the smallest raster saved to /Users/ddevi/Documents/SRC/smallest_raster_shapefile.shp
